In [1]:
import pandas as pd
import numpy as np  

In [2]:
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.table import Table, join
Vizier.ROW_LIMIT = -1
cat = Vizier.get_catalogs("J/MNRAS/505/5978")[0] # clusters + astrometria

dist = Table.read("BaumgartAndVasiliev_2023.csv")
dat = join(cat, dist, keys="Name", join_type="inner")

In [3]:
dat

Name,OName,RAJ2000,DEJ2000,pmRA,e_pmRA,pmDE,e_pmDE,corr,plx,e_plx,Rscale,Nstar,SimbadName,GEDR3_parallax_dist_kpc,GEDR3_parallax_dist_err_upper,GEDR3_parallax_dist_err_lower,GEDR3_kin_dist_kpc,GEDR3_kin_dist_err,HST_kin_dist_kpc,HST_kin_dist_err,Subdwarf_dist_kpc,Subdwarf_dist_err_upper,Subdwarf_dist_err_lower,Star_count_dist_kpc,Star_count_dist_err,Lit_dist_kpc,Lit_dist_err_upper,Lit_dist_err_lower,Mean_distance_kpc,Mean_distance_err_upper,Mean_distance_err_lower,N_Tot
,,deg,deg,mas / yr,mas / yr,mas / yr,mas / yr,,mas,mas,arcmin,,,,,,,,,,,,,,,,,,,,,
str12,str10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,str26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
AM 4,,209.090,-27.167,-0.291,0.445,-2.512,0.344,-0.36,0.079,0.343,0.39,8,AM 4,--,--,--,--,--,--,--,--,--,--,--,--,29.013,0.951,0.92,29.013,0.951,0.92,2
Arp 2,,292.184,-30.356,-2.331,0.031,-1.475,0.029,0.05,0.026,0.021,1.66,156,Arp 2,--,--,--,--,--,--,28.184,1.058,1.019,--,--,28.721,0.346,0.342,28.726,0.346,0.342,11.0,--
BH 140,,193.473,-67.177,-14.848,0.024,1.224,0.024,0.00,0.219,0.011,4.41,1197,Cl VDBH 140,4.808,0.258,0.233,--,--,--,--,--,--,--,--,--,--,4.808,0.258,0.233,1.0,--,--
BH 261,ESO 456-78,273.527,-28.635,3.566,0.043,-3.590,0.037,0.10,0.191,0.023,0.74,21,Cl VDBH 261,--,--,--,--,--,--,--,--,--,--,--,--,6.183,0.279,0.267,6.115,0.265,0.253,4
Crater,Laevens 1,174.067,-10.877,-0.059,0.125,-0.116,0.116,-0.23,0.003,0.142,0.51,15,NAME Crater Cluster,--,--,--,--,--,--,--,--,--,--,--,--,147.231,4.334,4.21,147.231,4.334,4.21,3
E 3,ESO 37-1,140.238,-77.282,-2.727,0.027,7.083,0.027,0.00,0.146,0.013,1.70,252,E 3,7.326,0.765,0.633,--,--,--,--,--,--,--,--,7.94,0.268,0.259,7.876,0.254,0.245,3.0,--
Eridanus,,66.185,-21.187,0.510,0.039,-0.301,0.041,-0.09,0.050,0.033,0.52,44,NAME Eridanus Star Cluster,--,--,--,--,--,--,--,--,--,--,--,--,84.723,2.937,2.839,84.684,2.936,2.838,1
FSR 1716,,242.625,-53.749,-4.354,0.033,-8.832,0.031,0.00,0.108,0.017,1.96,62,[FSR2007] 1716,--,--,--,--,--,--,--,--,--,--,--,--,7.321,0.275,0.265,7.431,0.27,0.26,4


In [26]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u

coords = SkyCoord(ra=dat["RAJ2000"], dec=dat["DEJ2000"], distance=dat["Mean_distance_kpc"]*u.kpc, frame="icrs")
print(coords)
gal = coords.galactic
l = gal.l.radian; b = gal.b.radian; d = coords.distance.to(u.kpc).value
x = d*np.cos(b)*np.cos(l)
y = d*np.cos(b)*np.sin(l)
z = d*np.sin(b)
P = np.c_[x,y,z]

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(209.09 , -2.7167e+01, 2.90130e+01),
     (292.184, -3.0356e+01, 3.46000e-01),
     (193.473, -6.7177e+01, 2.33000e-01),
     (273.527, -2.8635e+01, 6.11500e+00),
     (174.067, -1.0877e+01, 1.47231e+02),
     (140.238, -7.7282e+01, 2.54000e-01),
     ( 66.185, -2.1187e+01, 8.46840e+01),
     (242.625, -5.3749e+01, 7.43100e+00),
     (253.044, -4.7058e+01, 9.08200e+00),
     (262.8  , -3.9808e+01, 1.10850e+01),
     (261.785, -7.0930e+00, 2.65870e+01),
     (225.077, -8.2214e+01, 2.53000e-01),
     (316.729,  1.4984e+01, 6.17670e+01),
     (263.352, -3.3390e+01, 8.06100e+00),
     (275.832, -1.3669e+01, 5.46600e+00),
     (  6.024, -7.2081e+01, 4.52100e+00),
     ( 48.068, -5.5216e+01, 1.92000e-01),
     ( 78.528, -4.0047e+01, 1.34000e-01),
     ( 81.044, -2.4524e+01, 1.30780e+01),
     (102.248, -3.6005e+01, 1.70000e-01),
     (114.535,  3.8882e+01, 8.84710e+01),
     (138.013, -6.4863e+01, 1.00600e+01),
     ( 13.188, -2.6

In [11]:
c_mean = P.mean(axis=0)
R0_mean = np.linalg.norm(c_mean)

In [27]:
import numpy as np

# ---------------------------------------------------------------------
# 1. Implementação robusta do algoritmo de Weiszfeld (mediana geométrica)
# ---------------------------------------------------------------------
def geometric_median(X, eps=1e-6, itmax=10000):
    """
    Geometric median via Weiszfeld algorithm.
    X: array (n, d) com coordenadas dos pontos
    eps: tolerância para convergência
    """
    X = np.asarray(X, dtype=float)
    y = X.mean(axis=0)  # chute inicial

    for _ in range(itmax):
        D = np.linalg.norm(X - y, axis=1)

        # Caso especial: y coincide com algum ponto da amostra
        zero_mask = D < 1e-15
        if np.any(zero_mask):
            return X[zero_mask][0]

        W = 1.0 / np.clip(D, 1e-12, None)
        y1 = (X * W[:, None]).sum(axis=0) / W.sum()

        if np.linalg.norm(y1 - y) < eps:
            return y1
        
        y = y1

    return y  # retorna última estimativa, caso não converga totalmente


# --------------------------------------------------------------
# 2. Função para estimar o centro galáctico + bootstrap de R0
# --------------------------------------------------------------
def estimate_galactic_center(P, B=1000, seed=42):
    """
    P: array (N,3) com coordenadas cartesianas dos aglomerados.
    B: número de bootstraps.
    """
    # Mediana geométrica dos dados originais
    c_med = geometric_median(P)
    R0_med = np.linalg.norm(c_med)

    # Bootstrap
    rng = np.random.default_rng(seed)
    centers = np.empty((B, 3))

    N = len(P)
    for b in range(B):
        idx = rng.integers(0, N, N)  # bootstrap com reposição
        centers[b] = geometric_median(P[idx])

    # Distância solar → centro galáctico para cada bootstrap
    R0_boot = np.linalg.norm(centers, axis=1)

    # Intervalos de confiança (percentis)
    R0_ci = np.percentile(R0_boot, [16, 50, 84])

    return c_med, R0_med, R0_ci, centers, R0_boot


# --------------------------------------------------------------
# 3. Exemplo de uso (substitua pelo seu array P)
# --------------------------------------------------------------

c_med, R0_med, R0_ci, centers, R0_boot = estimate_galactic_center(P)

print("\n============= RESULTADOS =============")
print(f"Centro galáctico estimado (mediana geométrica):")
print(f"X = {c_med[0]:.3f} kpc")
print(f"Y = {c_med[1]:.3f} kpc")
print(f"Z = {c_med[2]:.3f} kpc\n")

print(f"Distância ao centro galáctico R0 = {R0_med:.3f} kpc")
print(f"Intervalos de confiança (16–50–84%):")
print(f"{R0_ci}")

print(f"\nIncertezas:")
print(f"+ {R0_ci[2] - R0_ci[1]:.3f}  /  - {R0_ci[1] - R0_ci[0]:.3f}  kpc")

# --------------------------------------------------------------


============= RESULTADOS =============
Centro galáctico estimado (mediana geométrica):
X = 0.258 kpc
Y = -0.007 kpc
Z = 0.002 kpc

Distância ao centro galáctico R0 = 0.259 kpc
Intervalos de confiança (16–50–84%):
[0.21139779 0.26165669 0.3204571 ]

Incertezas:
+ 0.059  /  - 0.050  kpc


In [23]:
print(f"R0 (centroide): {R0_mean:.3f} kpc")
print(f"R0 (mediana geométrica): {R0_med:.3f} kpc")
print(f"R0 (mediana geométrica, IC 68%): {R0_ci[1]:.3f} (+{R0_ci[2]-R0_ci[1]:.3f}/-{R0_ci[1]-R0_ci[0]:.3f}) kpc")

R0 (centroide): 3.390 kpc
R0 (mediana geométrica): 0.259 kpc
R0 (mediana geométrica, IC 68%): 0.262 (+0.059/-0.050) kpc


In [28]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_scatter3d(x=x, y=y, z=z, mode='markers',
marker=dict(size=3), name='GCs')
fig.add_scatter3d(x=[0], y=[0], z=[0], mode='markers',
marker=dict(size=6, symbol='diamond'), name='Sol')
fig.add_scatter3d(x=[c_med[0]], y=[c_med[1]], z=[c_med[2]], mode='markers',
marker=dict(size=6, symbol='x'), name='Centro (mediana)')
fig.update_layout(scene_aspectmode='data',
scene=dict(xaxis_title='x [kpc]',
yaxis_title='y [kpc]',
zaxis_title='z [kpc]'))
fig.show()